## DeepLearning Skills
- 모델의 overfitting 을 해결하기 위해?
  - Regularization
  - Early stopping

### 01 라이브러리 불러오기

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split

import numpy as np
import pandas as pd

from torchvision import transforms
from torchvision.datasets import FashionMNIST

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

classification_data_path = "./sample_data/"

validation_data_ratio = 0.2

random_state = 0

classifier_input_size = 28 * 28
classifier_hidden_size1 = 512
classifier_hidden_size2 = 128
classifier_hidden_size3 = 32
classifier_output_size = 10

l1_penalty = 1e-4
l2_penalty = 1e-4

dropout_rate = 0.2

learning_rate = 5e-3
epochs = 50
batch_size = 32

classification_criterion = nn.CrossEntropyLoss()

### 02 데이터 불러오는 클래스 정의

In [ ]:
class ClassificationDataLoader:
    def __init__(self, data_path, validation_data_ratio, batch_size):
        self.data_path = data_path
        self.validation_data_ratio = validation_data_ratio
        self.batch_size = batch_size
        self.transform = transforms.ToTensor()

    def __call__(self, flag):
        if flag == "train":
            dataset = FashionMNIST(
                self.data_path, train=True, download=True, transform=self.transform
            )

            train_data_size = int(len(dataset) * (1 - self.validation_data_ratio))
            validation_data_size = len(dataset) - train_data_size

            train_dataset, validation_dataset = random_split(
                dataset, [train_data_size, validation_data_size]
            )

            train_loader = DataLoader(
                train_dataset,
                batch_size=self.batch_size,
                shuffle=True,
                drop_last=True,
                num_workers=0,
            )
            validation_loader = DataLoader(
                validation_dataset,
                batch_size=self.batch_size,
                shuffle=True,
                drop_last=True,
                num_workers=0,
            )

            return train_loader, validation_loader
        else:
            test_dataset = FashionMNIST(
                self.data_path, train=False, download=True, transform=self.transform
            )
            test_loader = DataLoader(
                test_dataset,
                batch_size=self.batch_size,
                shuffle=False,
                drop_last=True,
                num_workers=0,
            )
            return test_loader





In [ ]:
classification_loader = ClassificationDataLoader(
    classification_data_path, validation_data_ratio, batch_size
)
classification_train_loader, classification_validation_loader = classification_loader(
    "train"
)
classification_test_loader = classification_loader("test")

### 03 Deep Learning 모델

In [ ]:
class DeepNeuralNetwork(nn.Module):
    def __init__(
        self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size
    ):
        super(DeepNeuralNetwork, self).__init__()
        self.input_size = input_size
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.hidden_size3 = hidden_size3
        self.output_size = output_size

        self.feature_extractor = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size1),
            nn.ReLU(),
            nn.Linear(self.hidden_size1, self.hidden_size2),
            nn.ReLU(),
            nn.Linear(self.hidden_size2, self.hidden_size3),
            nn.ReLU(),
        )
        self.output_layer = nn.Linear(self.hidden_size3, self.output_size)

    def forward(self, data):
        data = self.prepare_input(data)
        feature = self.feature_extractor(data)
        result = self.output_layer(feature)

        return result

    def prepare_input(self, data):
        flattened_data = data.view(data.size(0), -1)

        return flattened_data


class DropOutDNN(nn.Module):
    def __init__(
        self,
        input_size,
        dropout_rate,
        hidden_size1,
        hidden_size2,
        hidden_size3,
        output_size,
    ):
        super(DropOutDNN, self).__init__()
        self.input_size = input_size
        self.dropout_rate = dropout_rate
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.hidden_size3 = hidden_size3
        self.output_size = output_size

        self.feature_extractor = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size1),
            nn.ReLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.hidden_size1, self.hidden_size2),
            nn.ReLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.hidden_size2, self.hidden_size3),
            nn.ReLU(),
            nn.Dropout(self.dropout_rate),
        )
        self.output_layer = nn.Linear(self.hidden_size3, self.output_size)

    def forward(self, data):
        data = self.prepare_input(data)
        feature = self.feature_extractor(data)
        result = self.output_layer(feature)

        return result

    def prepare_input(self, data):
        flattened_data = data.view(data.size(0), -1)

        return flattened_data


classifier = DeepNeuralNetwork(
    classifier_input_size,
    classifier_hidden_size1,
    classifier_hidden_size2,
    classifier_hidden_size3,
    classifier_output_size,
).to(device)
dropout_classifier = DropOutDNN(
    classifier_input_size,
    dropout_rate,
    classifier_hidden_size1,
    classifier_hidden_size2,
    classifier_hidden_size3,
    classifier_output_size,
).to(device)


In [ ]:
classifier_optimizer = torch.optim.SGD(
    classifier.parameters(), lr=learning_rate, weight_decay=l2_penalty
)
dropout_classifier_optimizer = torch.optim.SGD(
    dropout_classifier.parameters(), lr=learning_rate, weight_decay=l2_penalty
)

### 04 Regularization(규제)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path="checkpoint.pt"):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(
                "EarlyStopping counter: {} out of {}".format(
                    self.counter, self.patience
                )
            )
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(
                "Test loss decreased ({:.6f}) --> {:.6f}. Saving model".format(
                    self.val_loss_min, val_loss
                )
            )
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


### 05 Test하기

In [ ]:
class Trainer:
    def __init__(
        self,
        model,
        train_data_loader,
        validation_data_loader,
        optimizer,
        criterion,
        epochs,
        device,
    ):
        self.model = model
        self.train_data_loader = train_data_loader
        self.validation_data_loader = validation_data_loader
        self.optimizer = optimizer
        self.criterion = criterion
        self.epochs = epochs
        self.device = device
        self.early_stopping = EarlyStopping()

    def train(self):
        self.model.train()

        for epoch in range(self.epochs):
            for data, label in self.train_data_loader:
                self.optimizer.zero_grad()
                data, label = data.to(self.device), label.to(self.device)
                result = self.model(data)
                loss = self.criterion(result, label)

                loss.backward()
                self.optimizer.step()

            val_loss = self.get_valid_loss()
            print(
                "epoch: {} / {}, validation loss: {}".format(
                    epoch, self.epochs, val_loss
                )
            )

            self.early_stopping(val_loss, self.model)
            if self.early_stopping.early_stop:
                print("Early stopping")
                break

    def train_l1_regularizer(self):
        self.model.train()

        for epoch in range(self.epochs):
            for data, label in self.train_data_loader:
                self.optimizer.zero_grad()
                data, label = data.to(self.device), label.to(self.device)
                result = self.model(data)

                loss = self.criterion(result, label)
                l1_loss = torch.FloatTensor([0]).to(device)
                for param in self.model.parameters():
                    l1_loss += torch.sum(torch.abs(param))

                loss = loss + l1_penalty * l1_loss

                loss.backward()
                self.optimizer.step()

            val_loss = self.get_valid_loss()
            print(
                "epoch: {} / {}, validation loss: {}".format(
                    epoch, self.epochs, val_loss
                )
            )

            self.early_stopping(val_loss, self.model)
            if self.early_stopping.early_stop:
                print("Early stopping")
                break

    def get_valid_loss(self):
        self.model.eval()
        valiation_loss = 0.0
        with torch.no_grad():
            for data, label in self.validation_data_loader:
                data, label = data.to(self.device), label.to(self.device)
                output = self.model(data)
                loss = self.criterion(output, label)
                valiation_loss += loss.item()

        return valiation_loss / len(self.validation_data_loader)


classifier_trainer = Trainer(
    classifier,
    classification_train_loader,
    classification_validation_loader,
    classifier_optimizer,
    classification_criterion,
    epochs,
    device,
)
classifier_trainer.train()

dropout_classifier_trainer = Trainer(
    dropout_classifier,
    classification_train_loader,
    classification_validation_loader,
    dropout_classifier_optimizer,
    classification_criterion,
    epochs,
    device,
)
dropout_classifier_trainer.train()
